In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.impute import KNNImputer
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.ensemble import RandomForestClassifier
import optuna
from sklearn.feature_selection import RFECV
from sklearn.linear_model import LogisticRegressionCV
import joblib
from mrmr import mrmr_classif
import xgboost as xgb

/Users/prathmesh/asep_project/venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Load both datasets
original = pd.read_csv("PCOS_data.csv")
new = pd.read_csv("pcos_dataset.csv")

# Set max columns to show is unlimited
pd.set_option('display.max_columns', None)

In [3]:
def preprocess(df):
    df = df.copy()  # avoid SettingWithCopyWarning

    # 1. Clean column names
    df.columns = df.columns.str.strip() \
                           .str.replace(' ', '_') \
                           .str.replace('(', '') \
                           .str.replace(')', '') \
                           .str.replace('.', '') \
                           .str.replace('-', '_') \
                           .str.replace('/', '_')
    df.rename(columns={'II____beta_HCGmIU_mL': 'II_beta_HCG'}, inplace=True)

    # 2. Drop irrelevant columns
    df.drop(columns=['Sl_No', 'Patient_File_No'], inplace=True, errors='ignore')
    df = df.loc[:, ~df.columns.str.startswith('Unnamed')]

    # 3. Merge Age columns
    if 'Age' not in df.columns and 'Age_yrs' in df.columns:
        df.rename(columns={'Age_yrs': 'Age'}, inplace=True)
    elif 'Age' in df.columns and 'Age_yrs' in df.columns:
        df['Age'] = df['Age'].fillna(df['Age_yrs'])
        df.drop(columns=['Age_yrs'], inplace=True)

    # 4. Merge PCOS diagnosis columns
    if 'PCOS_Diagnosis' in df.columns:
        df.rename(columns={'PCOS_Diagnosis': 'PCOS_Y_N'}, inplace=True)

    # 5. Handle missing values
    if 'Marraige_Status_Yrs' in df.columns:
        df.loc[:, 'Marraige_Status_Yrs'] = df['Marraige_Status_Yrs'].fillna(df['Marraige_Status_Yrs'].median())

    if 'Fast_food_Y_N' not in df.columns and 'Fast_food_YN' in df.columns:
        df.rename(columns={'Fast_food_YN': 'Fast_food_Y_N'}, inplace=True)
    if 'Fast_food_Y_N' in df.columns:
        df.loc[:, 'Fast_food_Y_N'] = df['Fast_food_Y_N'].fillna(df['Fast_food_Y_N'].mode()[0])

    # 6. Convert to numeric and fill missing values
    if 'II_beta_HCG' not in df.columns and 'II_beta_HCGmIU_mL' in df.columns:
        df.rename(columns={'II_beta_HCGmIU_mL': 'II_beta_HCG'}, inplace=True)
    if 'II_beta_HCG' in df.columns:
        df.loc[:, 'II_beta_HCG'] = pd.to_numeric(df['II_beta_HCG'], errors='coerce')
        df['II_beta_HCG'] = df['II_beta_HCG'].astype(float)
        df.loc[:, 'II_beta_HCG'] = df['II_beta_HCG'].fillna(df['II_beta_HCG'].median())

    if 'AMHng_mL' in df.columns:
        df.loc[:, 'AMHng_mL'] = pd.to_numeric(df['AMHng_mL'], errors='coerce')
        df['AMHng_mL'] = df['AMHng_mL'].astype(float)
        df.loc[:, 'AMHng_mL'] = df['AMHng_mL'].fillna(df['AMHng_mL'].median())

    return df

In [4]:
# Apply preprocessing
original_clean = preprocess(original)
new_clean = preprocess(new)

# Ensure consistent columns across both
all_columns = list(set(original_clean.columns).union(set(new_clean.columns)))

# Align both dataframes to same columns, fill missing with NaN
original_aligned = original_clean.reindex(columns=all_columns)
new_aligned = new_clean.reindex(columns=all_columns)

# Concatenate datasets
combined_df = pd.concat([original_aligned, new_aligned], ignore_index=True)

In [5]:
# Separate features and target
# X = combined_df.drop(columns=['PCOS_Y_N'])
X = combined_df.drop(columns=['PCOS_Y_N'])
y = combined_df['PCOS_Y_N']

# Initialize KNNImputer (e.g., with 5 nearest neighbors)
knn_imputer = KNNImputer(n_neighbors=5)

X = pd.DataFrame(knn_imputer.fit_transform(X), columns=X.columns)

# Sanitize feature names
X.columns = [str(col).replace(' ', '_')
                        .replace('"', '')
                        .replace("'", '')
                        .replace('[', '')
                        .replace(']', '')
                        .replace('{', '')
                        .replace('}', '')
                        .replace(':', '')
                        .replace(',', '')
                        for col in X.columns]

In [6]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X = pd.DataFrame(X_scaled, columns=X.columns)

In [7]:
selected_features = mrmr_classif(X=X, y=y, K=20)
print("Selected features:", selected_features)

100%|███████████████████████████████████████████████| 20/20 [00:00<00:00, 73.45it/s]

Selected features: ['Follicle_No_R', 'Menstrual_Irregularity', 'Weight_gainY_N', 'Antral_Follicle_Count', 'Testosterone_Levelng_dL', 'Follicle_No_L', 'Skin_darkening_Y_N', 'hair_growthY_N', 'CycleR_I', 'BMI', 'Fast_food_Y_N', 'AMHng_mL', 'Weight_Kg', 'PimplesY_N', 'Hipinch', 'Cycle_lengthdays', 'Waistinch', 'Hair_lossY_N', 'Avg_F_size_L_mm', 'Vit_D3_ng_mL']


In [8]:
X = X[selected_features]
X

,Follicle_No_R,Menstrual_Irregularity,Weight_gainY_N,Antral_Follicle_Count,Testosterone_Levelng_dL,Follicle_No_L,Skin_darkening_Y_N,hair_growthY_N,CycleR_I,BMI,Fast_food_Y_N,AMHng_mL,Weight_Kg,PimplesY_N,Hipinch,Cycle_lengthdays,Waistinch,Hair_lossY_N,Avg_F_size_L_mm,Vit_D3_ng_mL
0,-1.122920,0.656224,-1.087760,-0.416818,0.921052,-1.005156,-0.985893,-0.943971,-0.956554,-1.339000,1.307316,-0.905453,-1.586104,-1.401337,-0.862759,0.114870,-1.509019,-1.391155,1.248587,-0.103081
1,-0.496297,-0.745034,-1.087760,-0.551229,-0.683885,-1.005156,-0.985893,-0.943971,-0.956554,-0.159445,-1.446339,-1.033395,0.187682,-1.401337,-0.262582,0.114870,-0.852609,-1.391155,0.055793,0.102239
2,2.636818,1.123310,-1.087760,0.154429,0.412467,2.298768,-0.985893,-0.943971,-0.956554,-0.075192,1.307316,0.174942,0.518093,1.415596,0.337595,0.114870,0.460211,1.612166,1.248587,0.048354
3,-1.436231,0.656224,-1.087760,-0.215202,-0.414873,-1.335549,-0.985893,-0.943971,-0.956554,0.851601,-1.446339,-1.106843,0.187682,-1.401337,0.937772,0.114870,0.460211,-1.391155,0.055793,-0.027364
4,-0.809608,-0.277948,-1.087760,-0.248804,-0.029120,-1.005156,-0.985893,-0.943971,-0.956554,-1.170492,-1.446339,-0.860437,-0.942672,-1.401337,-0.562671,0.114870,-1.509019,1.612166,0.453391,0.020947
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1536,-0.370972,1.123310,-1.087760,0.927292,1.825542,-0.674764,0.196872,-0.309580,-0.956554,-1.528571,-0.895608,-0.135435,-1.499154,-1.401337,-2.123131,0.114870,-2.362353,-0.790491,0.811229,-0.056443
1537,-0.182985,1.123310,0.454317,-1.760928,-1.585330,0.250335,-0.394511,-0.309580,-0.355269,0.683094,-1.446339,-0.599341,0.196377,-0.274564,0.457630,-0.080080,0.591493,-0.189827,0.214832,-0.054213
1538,-0.370972,-1.212120,-0.059708,1.767361,-1.387377,-0.146136,-0.985893,-0.309580,-0.355269,0.556713,-0.344877,-0.143490,0.568524,-0.837951,0.517648,0.504769,0.460211,-0.189827,0.453391,-0.036951
1539,-0.809608,-1.212120,1.482369,-1.424901,1.820466,-0.344371,0.788254,-0.309580,0.847301,0.346078,1.307316,0.044631,0.648518,-0.274564,0.817736,0.114870,1.182263,-0.189827,0.135312,-0.061831


In [9]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.15, stratify=y, shuffle=True, random_state=8)

In [10]:
def objective(trial):
    
    param = {
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3, log=True),
        "n_estimators": trial.suggest_int("n_estimators", 100, 1000),
        "max_depth": trial.suggest_int("max_depth", 3, 12),
        "subsample": trial.suggest_float("subsample", 0.6, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.6, 1.0),
        "reg_alpha": trial.suggest_float("reg_alpha", 1e-3, 10.0, log=True),
        "reg_lambda": trial.suggest_float("reg_lambda", 1e-3, 10.0, log=True),
        "objective": "binary:logistic",
        "eval_metric": "logloss",
        "random_state": 8,
        "use_label_encoder": False,
    }

    train_pool = xgb.DMatrix(x_train, y_train)
    valid_pool = xgb.DMatrix(x_test, y_test)
    
    # Train model
    model = xgb.XGBClassifier(**param)
    model.fit(x_train, y_train,
              eval_set=[(x_test, y_test)],
              verbose=False)

    # Predict
    preds = model.predict(x_test)
    acc = accuracy_score(y_test, preds)

    return 1.0 - acc  # Optuna minimizes

In [11]:
study = optuna.create_study(direction='minimize')  # Because we return 1 - accuracy
study.optimize(objective, n_trials=50)  # Try 50 combinations (or more!)

print("Best hyperparameters:")
print(study.best_params)

print("Best accuracy:")
print(1 - study.best_value)

[I 2025-05-31 21:19:07,583] A new study created in memory with name: no-name-167082d4-5ecc-4d74-8618-4f6879dc8a93
[I 2025-05-31 21:19:08,233] Trial 0 finished with value: 0.03879310344827591 and parameters: {'learning_rate': 0.014697917087638446, 'n_estimators': 575, 'max_depth': 6, 'subsample': 0.7210979024447284, 'colsample_bytree': 0.89988407213509, 'reg_alpha': 0.003512649780635479, 'reg_lambda': 0.019140631628833987}. Best is trial 0 with value: 0.03879310344827591.
[I 2025-05-31 21:19:08,624] Trial 1 finished with value: 0.0431034482758621 and parameters: {'learning_rate': 0.09360433294120321, 'n_estimators': 355, 'max_depth': 11, 'subsample': 0.9219560138646214, 'colsample_bytree': 0.7355085055740559, 'reg_alpha': 0.0932284115096075, 'reg_lambda': 0.01901415811875373}. Best is trial 0 with value: 0.03879310344827591.
[I 2025-05-31 21:19:08,855] Trial 2 finished with value: 0.0431034482758621 and parameters: {'learning_rate': 0.08492137650458435, 'n_estimators': 153, 'max_depth':

Best hyperparameters:
{'learning_rate': 0.12608060183320494, 'n_estimators': 735, 'max_depth': 10, 'subsample': 0.6448944285234446, 'colsample_bytree': 0.8697910770454482, 'reg_alpha': 0.15198286950691367, 'reg_lambda': 1.1540636059453475}
Best accuracy:
0.978448275862069


In [12]:
# Get the data
best_model = xgb.XGBClassifier(**study.best_params)
best_model.fit(x_train, y_train,
          eval_set=[(x_test, y_test)],
          verbose=False)

# Predict
preds = best_model.predict(x_test)

# Compute final scores
acc = accuracy_score(y_test, preds)
precision = precision_score(y_test, preds, average='binary')
recall = recall_score(y_test, preds, average='binary')
f1 = f1_score(y_test, preds, average='binary')

print("\nFinal evaluation on test set:")
print(f"Accuracy: {acc * 100:.2f}")
print(f"Precision: {precision * 100:.2f}")
print(f"Recall: {recall * 100:.2f}")
print(f"F1 Score: {f1 * 100:.2f}")



Final evaluation on test set:
Accuracy: 96.98
Precision: 94.64
Recall: 92.98
F1 Score: 93.81
